### 1. Preparação dos Dados

#### a) Importando os arquivos CSV do dataset e as bibliotecas necessárias

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import matplotlib.pyplot as plt

# Caminho para a pasta dos datasets
dataset_path = "dataset"

# Lista todos os arquivos CSV na pasta
csv_files = [f for f in os.listdir(dataset_path) if f.endswith('.csv')]

# Cria um dicionário para armazenar os dataframes
dataframes = {}

# Importa todos os arquivos CSV como dataframes
for file in csv_files:
    df_name = file.replace('.csv', '')
    dataframes[df_name] = pd.read_csv(os.path.join(dataset_path, file))

customers_df = dataframes['customers']
order_items_df = dataframes['order_items']
sellers_df = dataframes['sellers']
products_df = dataframes['products']
product_category_name_translation_df = dataframes['product_category_name_translation']
order_payments_df = dataframes['order_payments']
orders_df = dataframes['orders']
order_reviews_df = dataframes['order_reviews']
geolocation_df = dataframes['geolocation']

#### b) Realizando a limpeza necessária 

In [ ]:
for name, df in dataframes.items():
    print(f"{name}:")
    print(df.isnull().sum())
    print("-" * 40)

In [ ]:
for name, df in dataframes.items():
    duplicates = df.duplicated().sum()
    print(f"{name}: {duplicates} duplicatas")

- Podemos perceber que nenhuma das colunas relevantes para a análise possuem valores nulos, ao mesmo tempo que somente as linhas de "geolocation"  
possuem duplicatas o que não é um problemas pois essa tabela contém as localizações dos vendedores e dos compradores que podem se repetir sem problemas  
- Também destaca-se a qualidade desses dados visto que ainda não foi necessária nenhuma modificação nos datasets

#### c) Fazendo a normalização das colunas

In [ ]:
order_items_df.describe()

In [ ]:
order_payments_df.describe()

- Dado o contexto o problema, os atributos que podem ser problemáticos numericamente e que talvez precisem ser normalizados podem estar relacionados a precificação  
mas analisando as estatísticas gerais das tabelas relacionadas a valores monetários vemos que existem outliers mas que condizem com o negócio

#### d) Criando um modelo relacional e conectando as tabelas adequadamente

- Como os problemas a serem resolvidos e as perguntas a serem respondidas diferem bastamtem optarei por realizar os JOINS "sob demanda", ou seja caso uma pergunta  
necessite de mais de uma tabela a conexão será realizada na resolução da pergunta. Caso seja possível reutilizarei as conexões para evitar processamento descessário  
  
- Para facilitar o entendimento dos dados produzi um diagrama Entidade-Relacionamento que servirá de guia para o projeto disponível no link:  
[diagrama](https://miro.com/app/board/uXjVI0LJsEA=/?share_link_id=70608596602)

![diagrama](diagrama.jpg)

### 2. Análise Exploratória de Dados 

#### a) volume de pedidos por mês / sazonalidade nas vendas

In [ ]:
# Converter a coluna de data para datetime
orders_df['order_purchase_timestamp'] = pd.to_datetime(orders_df['order_purchase_timestamp'])

# Criar coluna com número do mês
orders_df['mes'] = orders_df['order_purchase_timestamp'].dt.month

# Contar número de pedidos por mês (ignorando o ano)
total_por_mes = orders_df.groupby('mes').size().reset_index(name='num_pedidos')

# Adicionar nome dos meses
meses_nome = ['Janeiro', 'Fevereiro', 'Março', 'Abril', 'Maio', 'Junho', 
              'Julho', 'Agosto', 'Setembro', 'Outubro', 'Novembro', 'Dezembro']

total_por_mes['mes_nome'] = total_por_mes['mes'].apply(lambda x: meses_nome[x-1])

# Exibir resultado
display(total_por_mes[['mes_nome', 'num_pedidos']])


- Podemos afirmar que sim, existe sazonalidade nas vendas, o segundo quadrimestre possui a maior quantidade de pedidos e o terceiro quadrimestre possui um desempenho muito abaixo dos outros

#### b) distribuição do tempo de entrega dos pedidos

- Considerando o "tempo de entrega" o intervalo de dias entre a aprovação do pagamento e a chegada do produto

In [ ]:
# Converter as colunas para datetime, se ainda não estiverem
orders_df['order_approved_at'] = pd.to_datetime(orders_df['order_approved_at'])
orders_df['order_delivered_customer_date'] = pd.to_datetime(orders_df['order_delivered_customer_date'])

# Calcular a diferença em dias
orders_df['tempo_de_entrega'] = (orders_df['order_delivered_customer_date'] - orders_df['order_approved_at']).dt.days
orders_df['tempo_de_entrega'] = orders_df['tempo_de_entrega'].fillna(0).astype(int)

display((orders_df.groupby('tempo_de_entrega')
                    .size().
                    reset_index(name='num_pedidos'))
                    .sort_values(by='num_pedidos', ascending=False)
                    .head(25))

media_tempo_entrega = (orders_df['tempo_de_entrega'].sum() / orders_df['tempo_de_entrega'].count())
print(f"Média de tempo de entrega (em dias): {media_tempo_entrega:.2f}")

- Temos que a maioria dos pedidos é entregue entre 5 a 20 dias com um número grande de entregas realizadas no mesmo dia, além disso temos em média  
11 dias para que o produto seja entregue

#### c) relação entre o valor do frete e a distância de entrega

- Para resolver essa pergunta utilizarei as tabelas "orders", "customers", "order_items" e "sellers" para obter respectivamente o "order_id", ("customer_city" e "customer_state"), "freight_value", ("seller_city" e "seller_state") e com esses dados poderei obter informações importantes sobre a distância de entrega

In [ ]:
orders_customers = orders_df.merge(customers_df, on='customer_id', how='inner')[['order_id', 'customer_city', 'customer_state', 'customer_unique_id']]
display(orders_customers.head())

orders_customers_items = orders_customers.merge(order_items_df[['order_id', 'seller_id', 'freight_value']], on='order_id', how='inner')
display(orders_customers_items.head())

orders_customers_items_sellers = orders_customers_items.merge(
    sellers_df[['seller_id', 'seller_city', 'seller_state']],
    on='seller_id',
    how='left'
)

orders_customers_items_sellers = orders_customers_items_sellers.drop(columns=['seller_id'])
display(orders_customers_items_sellers.head())

- Dividirei os pedidos entre "Mesma Cidade", "Mesmo Estado", e "Mesma Região". Assim podemos ter uma visão mais geral das distâncias

In [ ]:
orders_customers_items_sellers['same_city'] = np.where(
    orders_customers_items_sellers['customer_city'].str.lower() == orders_customers_items_sellers['seller_city'].str.lower(),
    'yes', 'no'
)
orders_customers_items_sellers['same_state'] = np.where(
    orders_customers_items_sellers['customer_state'].str.lower() == orders_customers_items_sellers['seller_state'].str.lower(),
    'yes', 'no'
)

estado_para_regiao = {
    'AC': 'NO', 'AP': 'NO', 'AM': 'NO', 'PA': 'NO', 'RO': 'NO', 'RR': 'NO', 'TO': 'NO',
    'AL': 'NE', 'BA': 'NE', 'CE': 'NE', 'MA': 'NE', 'PB': 'NE', 'PE': 'NE', 'PI': 'NE', 'RN': 'NE', 'SE': 'NE',
    'DF': 'CO', 'GO': 'CO', 'MT': 'CO', 'MS': 'CO',
    'ES': 'SE', 'MG': 'SE', 'RJ': 'SE', 'SP': 'SE',
    'PR': 'SU', 'RS': 'SU', 'SC': 'SU'
}

orders_customers_items_sellers['customer_region'] = orders_customers_items_sellers['customer_state'].map(
    estado_para_regiao
).fillna('Outros')

orders_customers_items_sellers['seller_region'] = orders_customers_items_sellers['seller_state'].map(
    estado_para_regiao
).fillna('Outros')

orders_customers_items_sellers['same_region'] = np.where(
    orders_customers_items_sellers['customer_region'] == orders_customers_items_sellers['seller_region'],
    'yes', 'no'
)

orders_customers_items_sellers 

- Com todas essas informações finalmente podemos analisar o valor do frete 

In [ ]:
# Cálculo das médias de frete por localização
media_frete_mesma_cidade = orders_customers_items_sellers.loc[
    orders_customers_items_sellers['same_city'] == 'yes', 'freight_value'
].mean()

media_frete_mesmo_estado = orders_customers_items_sellers.loc[
    orders_customers_items_sellers['same_state'] == 'yes', 'freight_value'
].mean()

media_frete_mesma_regiao = orders_customers_items_sellers.loc[
    orders_customers_items_sellers['same_region'] == 'yes', 'freight_value'
].mean()

media_frete_diferente_regiao = orders_customers_items_sellers.loc[
    orders_customers_items_sellers['same_region'] == 'no', 'freight_value'
].mean()

# Exibição organizada dos resultados
print("🏷️  Média do valor do frete por localização:")
print("-----------------------------------------------")
print(f"🚚 Entregas na mesma cidade:        R$ {media_frete_mesma_cidade:.2f}")
print(f"🚛 Entregas no mesmo estado:        R$ {media_frete_mesmo_estado:.2f}")
print(f"🧭 Entregas na mesma região:        R$ {media_frete_mesma_regiao:.2f}")
print(f"🗺️  Entregas em regiões diferentes: R$ {media_frete_diferente_regiao:.2f}")


#### d) categorias de produtos mais vendidos em termos de faturamento

Considerei os "produtos mais vendidos em termos de faturamento" como os "produtos que geraram mais receita". Para isso não utilizarei o "payment_value" pois contém incluso o valor do frete  
uma opção mais "limpa" é o ("price" x "order_item_id") pois obtemos o valor real do pedido

In [ ]:
order_items_products = order_items_df.merge(products_df[['product_id', 'product_category_name']], on='product_id', how='inner')[['order_id', 'product_category_name', 'price', 'order_item_id']]
order_items_products['real_price'] = order_items_products['price'] * order_items_products['order_item_id']

order_items_products_name_realprice = order_items_products.drop(columns=['price', 'order_item_id', 'order_id'])
produtos_mais_vendidos = order_items_products_name_realprice.groupby(['product_category_name']).sum().reset_index()
produtos_vendidos_ordenados = produtos_mais_vendidos.rename(columns={'real_price': 'total_revenue'}).sort_values(by=['total_revenue'], ascending=False)

display(produtos_vendidos_ordenados.head(25))


- Temos então as categorias mostradas acima como aquelas que geram maior faturamento

#### e) estados brasileiros com o maior valor médio de pedido

- como "valor de pedido" é um termo mais abrangente desta vez podemos usar diretamente o "payment value"  
- considerarei como "estado do pedido "o "customer_state", ou seja onde o pedido chegará
- vou reutilizar o "orders_customers" do item c)

In [ ]:
# orders_customers = orders_df.merge(customers_df, on='customer_id', how='inner')[['order_id', 'customer_city', 'customer_state']]

orders_customers_no_city = orders_customers.drop(columns=['customer_city'])
orders_customers_payments = orders_customers_no_city.merge(order_payments_df[['order_id', 'payment_value']], on='order_id', how='inner')
orders_customers_payments = orders_customers_payments.drop(columns=['order_id'])
orders_customers_payments = orders_customers_payments.drop(columns=['customer_unique_id'])
valor_medio_por_estado = orders_customers_payments.groupby(['customer_state']).mean().reset_index()
valor_medio_por_estado = valor_medio_por_estado.rename(columns={'payment_value': 'payment_mean'}).sort_values(by=['payment_mean'], ascending=False)
valor_medio_por_estado['payment_mean'] = valor_medio_por_estado['payment_mean'].round(2)

display(valor_medio_por_estado.head(25))

- Temos então os estados gerados acima como aqueles com maior valor médio de pedido 

### 3. Solução de Problemas de Negócio 

#### a) **Análise de Retenção**


- Antes de realizar qualquer análise sobre os cliente recorrentes primeiro precisamos saber quantos são os clientes recorrentes 

In [ ]:
orders_customers.head(10)
# Quantidade total de clientes únicos
total_clientes_unicos = orders_customers['customer_unique_id'].nunique()
print(f"Total de customer_unique_id diferentes: {total_clientes_unicos}")

# Quantidade de clientes que aparecem apenas uma vez
clientes_uma_vez = orders_customers['customer_unique_id'].value_counts()
clientes_unicos_uma_vez = (clientes_uma_vez == 1).sum()
print(f"Quantidade de customer_unique_id que aparece apenas uma vez: {clientes_unicos_uma_vez}")
print(f"Quantidade de clientes recorrentes: {total_clientes_unicos - clientes_unicos_uma_vez}")
print(f"Proporção de clientes recorrentes: {(total_clientes_unicos - clientes_unicos_uma_vez) / total_clientes_unicos:.2%}")


- Pela saída do código acima podemos ver que somente **3.12%** dos clientes são recorrentes, correspondendo a **2997** clientes. Essa taxa é muito baixa, demonstrando que  
apenas uma pequena parte dos clientes volta a realizar compras  

- Munidos dessas informações tentaremos responder a pergunta "Quem são os clientes recorrentes" ao obtermos os **estados** de onde eles vem e qual a **categoria de produto** que  
eles mais compram 

In [ ]:
orders_customers.head(10)
clientes_recorrentes_ids = clientes_uma_vez[clientes_uma_vez > 1].index

orders_customers_recorrentes = orders_customers[orders_customers['customer_unique_id'].isin(clientes_recorrentes_ids)]
display(orders_customers_recorrentes.describe())

orders_customers_recorrentes_items = orders_customers_recorrentes.merge(order_items_df[['order_id', 'product_id']], on='order_id', how='inner')
orders_customers_recorrentes_items_products = orders_customers_recorrentes_items.merge(products_df[['product_id', 'product_category_name']], on='product_id', how='inner')
clientes_recorrentes_clean = orders_customers_recorrentes_items_products.drop(columns=['customer_city', 'product_id'])
clientes_recorrentes_clean.head()

# Gráfico dos estados mais recorrentes entre clientes recorrentes
plt.figure(figsize=(10, 5))
clientes_recorrentes_clean['customer_state'].value_counts().head(15).plot(kind='bar', color='skyblue')
plt.title('Top Estados dos Clientes Recorrentes')
plt.xlabel('Estado')
plt.ylabel('Quantidade de Pedidos')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Gráfico das categorias de produto mais recorrentes entre clientes recorrentes
plt.figure(figsize=(12, 5))
clientes_recorrentes_clean['product_category_name'].value_counts().head(15).plot(kind='bar', color='orange')
plt.title('Top Categorias de Produto dos Clientes Recorrentes')
plt.xlabel('Categoria de Produto')
plt.ylabel('Quantidade de Pedidos')
plt.xticks(rotation=75)
plt.tight_layout()
plt.show()

- Com essas informações podemos tirar insights valiosos para aumentar a fidelização de clientes  

- Poderia-se por exemplo investir mais no marketing de produtos de "cama_mesa_banho", "moveis_decoracao", "esportes_lazer" que lideram as recompras  

- Também poderiamos aumentar o número de anúncios para clientes do sudeste (principalmente sao paulo) que possuem uma fidelização maior


#### 2. **Predição de Atraso**

- Será criada uma nova coluna denominada "atraso" que retornará 1 caso o produto esteja atrasado e 0 caso contrário. Para criar essa coluna comparei os  
atributos "order_delivered_customer_date" e "order_estimated_delivery_date", caso a primeira passe da segunda temos um atraso. 

- Apesar de ser um problema de decisão binária não utilizarei a **regressão linear** pois os dados não seguem um padrão linear, por isso optarei pela **random forest**  
devido ao seu caráter mais robusto em relação as relações complexas dos dados  

- Para treinar o modelo utilizarei toda a tabela "orders", toda a tabela "products", os atributos 'seller_city' e 'seller_state' da tabela "sellers", os atributos  
'customer_city' e 'customer_state' da tabela "customers", o 'review_score' da tabela "order_reviews", e o 'payment_value' da tabela "order_payments"

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Converter as colunas para datetime, se ainda não estiverem
orders_df['order_estimated_delivery_date'] = pd.to_datetime(orders_df['order_estimated_delivery_date'])
orders_df['order_delivered_carrier_date' ] = pd.to_datetime(orders_df['order_delivered_carrier_date'])

# Criar a coluna "atraso"
orders_df['atraso'] = (
    (orders_df['order_delivered_customer_date'] > orders_df['order_estimated_delivery_date'])
    & orders_df['order_delivered_customer_date'].notnull()
    & orders_df['order_estimated_delivery_date'].notnull()
).astype(int)

orders_df.head(30)

orders_items = orders_df.merge(order_items_df[['order_id','seller_id', 'product_id']], on='order_id', how='inner')
orders_items_products = orders_items.merge(products_df, on='product_id', how='inner')
orders_items_products_sellers = orders_items_products.merge(sellers_df[['seller_id', 'seller_city','seller_state']], on='seller_id', how='inner')
orders_items_products_sellers_customers = orders_items_products_sellers.merge(customers_df[['customer_id','customer_unique_id', 'customer_city', 'customer_state']], on='customer_id', how='inner')
orders_items_products_sellers_customers_reviews = orders_items_products_sellers_customers.merge(order_reviews_df[['order_id', 'review_score']], on='order_id', how='inner')


le_seller_city = LabelEncoder()
le_seller_state = LabelEncoder()
le_customer_city = LabelEncoder()
le_customer_state = LabelEncoder()

orders_items_products_sellers_customers_reviews['seller_city_encoded'] = le_seller_city.fit_transform(
    orders_items_products_sellers_customers_reviews['seller_city'].astype(str)
)
orders_items_products_sellers_customers_reviews['seller_state_encoded'] = le_seller_state.fit_transform(
    orders_items_products_sellers_customers_reviews['seller_state'].astype(str)
)
orders_items_products_sellers_customers_reviews['customer_city_encoded'] = le_customer_city.fit_transform(
    orders_items_products_sellers_customers_reviews['customer_city'].astype(str)
)
orders_items_products_sellers_customers_reviews['customer_state_encoded'] = le_customer_state.fit_transform(
    orders_items_products_sellers_customers_reviews['customer_state'].astype(str)
)

orders_items_products_sellers_customers_reviews.head()

orders_items_products_sellers_customers_reviews.dtypes

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Seleciona as colunas de features (removendo variáveis que não serão usadas diretamente)
features = orders_items_products_sellers_customers_reviews.drop(columns=['atraso', 'order_id', 'customer_id', 'product_id', 'seller_id', 'customer_unique_id'])
# Remove colunas que não são do tipo int, float ou bool
features = features.select_dtypes(include=['int', 'float', 'bool'])

# Variável alvo
target = orders_items_products_sellers_customers_reviews['atraso']

# Divisão em treino e teste (80% treino, 20% teste)
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42, stratify=target)

features.dtypes

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt

# Ajuste do modelo Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)

# Previsão no conjunto de teste
y_pred = rf.predict(X_test)

# Avaliação do modelo
print("Acurácia:", accuracy_score(y_test, y_pred))


In [ ]:
# Matriz de confusão visual
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=rf.classes_, yticklabels=rf.classes_)
plt.title('Matriz de Confusão')
plt.xlabel('Previsto')
plt.ylabel('Real')
plt.tight_layout()
plt.show()

In [ ]:
# Relatório de Classificação
print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred))

In [ ]:
# Análise dos resultados - Importância das variáveis
importances = rf.feature_importances_
feature_names = features.columns
importances_df = pd.DataFrame({'feature': feature_names, 'importance': importances})
importances_df = importances_df.sort_values(by='importance', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x='importance', y='feature', data=importances_df.head(10), palette='viridis')
plt.title('Top 10 Features mais importantes para o modelo')
plt.xlabel('Importância')
plt.ylabel('Feature')
plt.tight_layout()
plt.show()

- Tivemos uma ótima performace do modelo, e como esperado a variável **tempo_de_entrega** foi a mais importante para a decisão 

#### 3. **Segmentação de Clientes**

- Essa clusterização buscará avaliar comportamentos de compra, por isso dados relevantes para a clusterização: "order_id", "categoria do produto", "payment_value", "recorrencia"  

- Reutilizarei o código em **1 Análise de Retenção** para criar uma coluna que determina se o cliente é fidelizado ou não

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import pandas as pd
import seaborn as sns

#clientes_recorrentes_ids

segmentacao_de_clientes = orders_items_products_sellers_customers[['order_id', 'product_category_name', 'customer_unique_id']]
segmentacao_de_clientes = segmentacao_de_clientes.merge(order_payments_df[['order_id', 'payment_value']], on='order_id', how='inner')


segmentacao_de_clientes['loyal_customer'] = segmentacao_de_clientes['customer_unique_id'].isin(clientes_recorrentes_ids).astype(int)
segmentacao_de_clientes.head()
segmentacao_de_clientes['loyal_customer'].value_counts()
segmentacao_de_clientes = segmentacao_de_clientes.drop(columns=['order_id'])


# Agrupa dados por cliente
agg_clientes = segmentacao_de_clientes.groupby('customer_unique_id').agg({
    'payment_value': 'mean',  # valor médio pago
    'product_category_name': lambda x: x.mode()[0] if not x.mode().empty else 'unknown',  # categoria mais comprada
    'loyal_customer': 'max'  # se é recorrente (0 ou 1)
}).reset_index()

# Transforma categoria em variáveis dummies
agg_clientes_dummies = pd.get_dummies(agg_clientes, columns=['product_category_name'], drop_first=True)



In [ ]:

# Normaliza os dados (exceto customer_id)
scaler = StandardScaler()
X = scaler.fit_transform(agg_clientes_dummies.drop(columns=['customer_unique_id']))

# Aplica KMeans
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
agg_clientes_dummies['cluster'] = kmeans.fit_predict(X)

# Junta o cluster de volta na segmentacao_de_clientes, se quiser
segmentacao_de_clientes = segmentacao_de_clientes.merge(
    agg_clientes_dummies[['customer_unique_id', 'cluster']],
    on='customer_unique_id',
    how='left'
)

# Visualiza a distribuição de clientes por cluster
print(agg_clientes_dummies['cluster'].value_counts())


In [ ]:

import matplotlib.pyplot as plt

# Visualização dos clusters em relação ao valor médio pago e fidelidade
plt.figure(figsize=(10, 6))
sns.boxplot(x='cluster', y='payment_value', data=segmentacao_de_clientes, showfliers=False)
plt.title('Distribuição do Valor Médio Pago por Cluster')
plt.xlabel('Cluster')
plt.ylabel('Valor Médio Pago')
plt.show()

plt.figure(figsize=(8, 5))
sns.countplot(x='cluster', hue='loyal_customer', data=segmentacao_de_clientes, palette='Set2')
plt.title('Distribuição de Clientes Fidelizados por Cluster')
plt.xlabel('Cluster')
plt.ylabel('Quantidade de Pedidos')
plt.legend(title='Fidelizado (1=Sim)')
plt.show()

# Análise dos clusters
for cluster_id in sorted(segmentacao_de_clientes['cluster'].unique()):
    grupo = segmentacao_de_clientes[segmentacao_de_clientes['cluster'] == cluster_id]
    pct_fidelizados = grupo['loyal_customer'].mean() * 100
    valor_medio = grupo['payment_value'].mean()
    categoria_top = grupo['product_category_name'].mode()[0] if not grupo['product_category_name'].mode().empty else 'unknown'
    print(f"Cluster {cluster_id}:")
    print(f"  - % Clientes Fidelizados: {pct_fidelizados:.1f}%")
    print(f"  - Valor Médio Pago: R$ {valor_medio:.2f}")
    print(f"  - Categoria Mais Comprada: {categoria_top}")
    # Estratégias de marketing sugeridas
    if pct_fidelizados > 50 and valor_medio > segmentacao_de_clientes['payment_value'].mean():
        print("  Estratégia: Investir em programas de fidelidade premium e ofertas exclusivas para manter e aumentar o ticket médio.")
    elif pct_fidelizados > 50:
        print("  Estratégia: Oferecer recompensas por recorrência e descontos progressivos para estimular compras de maior valor.")
    elif valor_medio > segmentacao_de_clientes['payment_value'].mean():
        print("  Estratégia: Focar em campanhas de upsell/cross-sell e benefícios para primeira recompra.")
    else:
        print("  Estratégia: Campanhas de aquisição, cupons de boas-vindas e comunicação focada em benefícios do produto.")
    print("-" * 60)



#### 4. **Análise de Satisfação**

- Para resolver essa questão utilizarei a **correlação de pearson** entre os atributos escolhidos, no entanto nem todos os atributos são númericos(condicao necessária para aplicar a relação), por isso a coluna "product_category_name" será codificada para valores númericos.  

- Vale ressaltar que este tipo de prática está longe de ser ideal, mas "resolve" o nosso problema

In [ ]:
from sklearn.preprocessing import LabelEncoder

#nota de avaliação, categoria do produto, tempo de entrega, valor do pedido

order_items_products_clean = order_items_products.drop(columns=['real_price', 'order_item_id'])
items_products_orders = order_items_products_clean.merge(orders_df[['order_id', 'tempo_de_entrega']], on='order_id', how='inner')
items_products_orders_reviews = items_products_orders.merge(order_reviews_df[['order_id', 'review_score']], on='order_id', how='inner')
items_products_orders_reviews.head()

le = LabelEncoder()
items_products_orders_reviews['product_category_encoded'] = le.fit_transform(
    items_products_orders_reviews['product_category_name'].astype(str)
)

items_products_orders_reviews = items_products_orders_reviews.drop(columns=['product_category_name', 'order_id'])

# Calculando a correlação de Pearson
correlacao = items_products_orders_reviews.corr(method='pearson')

# Exibindo o heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(correlacao, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Heatmap de Correlação de Pearson')
plt.show()



- Podemos perceber que **baixa** é a relação entre os atributos escolhidos e a satisfação do cliente, somente o tempo de entrega que possui uma correlação negativa  
pequena sobre a satisfação, ou seja quanto menor o tempo de entrega maior a satisfação do cliente 

### 4. Visualização e Dashboards

#### 1. Evolução das vendas ao longo do tempo

- O estado de referência será o estado em que a venda foi efetuada, ou seja "seller_state"

In [ ]:

import matplotlib.pyplot as plt
import seaborn as sns

# Agrupar por mês e contar número de vendas

orders_items_products_sellers['mes_ano'] = orders_items_products_sellers['order_purchase_timestamp'].dt.to_period('M')
vendas_por_mes = orders_items_products_sellers[['order_id', 'seller_id', 'order_purchase_timestamp', 'product_category_name', 'seller_state', 'tempo_de_entrega','mes_ano']].groupby('mes_ano').size().reset_index(name='num_vendas')
vendas_por_mes['mes_ano'] = vendas_por_mes['mes_ano'].astype(str)

# Plotar o gráfico
plt.figure(figsize=(12, 5))
sns.lineplot(data=vendas_por_mes, x='mes_ano', y='num_vendas', marker='o')
plt.title('Evolução das Vendas ao Longo do Tempo')
plt.xlabel('Mês/Ano')
plt.ylabel('Número de Vendas')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()


- infelizmente os widgets de filtro não estão funcionando mas o gráfico saiu de maneira correta

#### 2. Concentração de vendas por região/estado do Brasil

In [ ]:
import folium
from folium.plugins import HeatMap


orders_customers = orders_df[['order_id', 'customer_id']].merge(customers_df[['customer_id', 'customer_zip_code_prefix']], on='customer_id', how='inner')

orders_customers_geolocation = orders_customers.merge(geolocation_df[['geolocation_zip_code_prefix', 'geolocation_lat', 'geolocation_lng']], 
                                                      left_on='customer_zip_code_prefix', right_on='geolocation_zip_code_prefix', how='inner')

vendas_com_latitude_longitude = orders_customers_geolocation[['order_id', 'geolocation_lat', 'geolocation_lng']]

display(vendas_com_latitude_longitude.head(30))

# Amostra para evitar sobrecarga de memória (opcional)
sample_size = 50000
vendas_sample = vendas_com_latitude_longitude.sample(n=sample_size, random_state=42)

# Criar o mapa centralizado no Brasil
m = folium.Map(location=[-14.2350, -51.9253], zoom_start=4)

# Adicionar o HeatMap
HeatMap(
    data=vendas_sample[['geolocation_lat', 'geolocation_lng']].values,
    radius=8,
    blur=10,
    min_opacity=0.2,
    max_zoom=1
).add_to(m)

m

#### 3. Relação avaliação do cliente x tempo de entrega

In [ ]:
orders_reviews = orders_df[['order_id', 'tempo_de_entrega']].merge(
    order_reviews_df[['order_id', 'review_score']], on='order_id', how='inner')

orders_reviews.head(10)

plt.figure(figsize=(12, 6))
sns.boxplot(x='review_score', y='tempo_de_entrega', data=orders_reviews)
plt.ylim(0, 100)
plt.title('Tempo de Entrega por Nota de Avaliação (Review Score)')
plt.xlabel('Nota de Avaliação (review_score)')
plt.ylabel('Tempo de Entrega (dias)')
plt.grid(True)
plt.show()

plt.figure(figsize=(12, 6))
sns.violinplot(x='review_score', y='tempo_de_entrega', data=orders_reviews, inner='quartile')
plt.ylim(0, 100)
plt.title('Distribuição do Tempo de Entrega por Nota de Avaliação')
plt.xlabel('Nota de Avaliação (review_score)')
plt.ylabel('Tempo de Entrega (dias)')
plt.grid(True)
plt.show()

plt.figure(figsize=(10, 6))
sns.scatterplot(x='tempo_de_entrega', y='review_score', data=orders_reviews, alpha=0.1)
plt.xlim(0, 100)
plt.title('Dispersão: Tempo de Entrega vs Nota de Avaliação')
plt.xlabel('Tempo de Entrega (dias)')
plt.ylabel('Nota de Avaliação (review_score)')
plt.grid(True)
plt.show()


- É possível notar que pouca é a diferença entre o tempo de entrega para as notas 3, 4 e 5. No entanto, para as notas 1 e 2 podemos perceber um crescimento do tempo de entrega  

- Principalmente a partir do gráfico de violino podemos perceber a relação entre as duas variáveis


#### 4. Dashboard de análise dos vendedores

- Como temos 3090 vendedores distintos (ao realizarmos o merge das tabelas) vamos diminuir os escopo da nossa análise para os 50 "maiores" vendedores que são os 50  
com a maior quantidade de pedidos. Isso será feito por conta da relevância desses vendedores no dataset.  


- Poderiamos mostrar os top 50 vendedores considerando somente as "melhores avaliações" no entanto isso favoreceria vendedores com poucas vendas ou seja teriamos as "melhores avliações" de vendedores que venderam 1 a 5 produtos que é um volume irrelevante para o dataset. O mesmo vale para o tempo de entrega.

- Infelizmente não temos o nome dos vendedores o que torna os gráficos "poluídos". Poderiamos atribuir um "apelido" a cada vendedor ganhariamos legibilidade em detrimento da informação de quem são de fato os top 50 

In [ ]:
# volume de vendas, satisfação do cliente e tempo de entrega
analise_dos_vendedores = orders_items_products_sellers[['order_id', 'seller_id', 'tempo_de_entrega' ]]
analise_dos_vendedores = analise_dos_vendedores.merge(order_reviews_df[['order_id', 'review_score']], on='order_id', how='inner')

# Quantidade de vendedores distintos
num_sellers = analise_dos_vendedores['seller_id'].nunique()
print(f"Número de seller_id distintos: {num_sellers}")

top_25 = analise_dos_vendedores['seller_id'].value_counts().head(25)
plt.figure(figsize=(14, 6))
sns.barplot(x=top_25.index, y=top_25.values, palette='viridis')
plt.title('Top 25 Vendedores com Mais Vendas')
plt.xlabel('seller_id')
plt.ylabel('Quantidade de Vendas')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

# Calcular a média das reviews para os top 25 vendedores
media_reviews_top_25 = (
    analise_dos_vendedores[analise_dos_vendedores['seller_id'].isin(top_25.index)]
    .groupby('seller_id')['review_score']
    .mean()
    .loc[top_25.index]  # mantém a ordem dos top_25
)

# Ordenar em ordem decrescente de média das avaliações
media_reviews_top_25_sorted = media_reviews_top_25.sort_values(ascending=False)

plt.figure(figsize=(14, 6))
sns.barplot(x=media_reviews_top_25_sorted.index, y=media_reviews_top_25_sorted.values, palette='coolwarm')
plt.title('Top 25 Vendedores - Média das Avaliações dos Clientes (Ordem Decrescente)')
plt.xlabel('seller_id')
plt.ylabel('Média das Avaliações (review_score)')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

# Calcular a média do tempo de entrega para os top 25 vendedores
media_tempo_entrega_top_25 = (
    analise_dos_vendedores[analise_dos_vendedores['seller_id'].isin(top_25.index)]
    .groupby('seller_id')['tempo_de_entrega']
    .mean()
    .loc[top_25.index]  # mantém a ordem dos top_25
)
# Ordenar em ordem crescente de média do tempo de entrega
media_tempo_entrega_top_25_sorted = media_tempo_entrega_top_25.sort_values(ascending=True)
plt.figure(figsize=(14, 6))
sns.barplot(x=media_tempo_entrega_top_25_sorted.index, y=media_tempo_entrega_top_25_sorted.values, palette='magma')
plt.title('Top 25 Vendedores - Média do Tempo de Entrega (Ordem Crescente)')
plt.xlabel('seller_id')
plt.ylabel('Média do Tempo de Entrega (dias)')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

